TASK 4

In [3]:
#Importing Libraries
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline

In [4]:
#Importaing Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = CIFAR10(root='data/', download=True, transform=transform)
test_set = CIFAR10(root='data/', train=False, transform=transform)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1125)>

Data Analysis

In [ ]:
#Inspecting training dataset
size_trainset = len(train_set)
print(f"Length of training dataset is {size_trainset}")

#Inspecting testing dataset
size_testset = len(test_set)
print(f"Length of testing dataset is {size_testset}")

In [ ]:
#Inspecting output classes
classes = train_set.classes
classes

In [ ]:
total_classes = len(classes)
print(f"Total number of output classes are {total_classes} ")

In [ ]:
#Identifying shape of image tensor
image, label = train_set[0]
image_shape = image.shape
image_shape

In [ ]:
#As imshow expects images to be structured as (rows, columns, channels) values for RGB data
#We will rearrange original image tensor according to the desired ordering 
image, label = train_set[6]
plt.imshow(image.permute((1, 2, 0)))
print('Label id:', label)
print('Label (corresponding text):', classes[label])

Preparing data for processing

In [ ]:
# percentage of training set to use as validation
size_val = 0.2

# obtain training indices that will be used for validation
size_trainset = len(train_set)
indices = list(range(size_trainset))
np.random.shuffle(indices)
split = int(np.floor(size_val * size_trainset))
train_idx, valid_idx = indices[split:], indices[:split]

# defining sampler
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

#loading training data for cnn in batches using DataLoader
train_loader_cnn = torch.utils.data.DataLoader(train_set, batch_size=20, sampler=train_sampler, num_workers=0)
valid_loader_cnn = torch.utils.data.DataLoader(train_set, batch_size=20, sampler=valid_sampler, num_workers=0)
test_loader_cnn = torch.utils.data.DataLoader(test_set, batch_size=10000, num_workers=0)

In [ ]:
#loading training data for ann in batches using DataLoader
#setting seed to 80 for generating random numbers
torch.manual_seed(80)
train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
test_loader = DataLoader(test_set, batch_size=500, shuffle=False)

Building ANN Model

In [ ]:
class ANN(nn.Module):
    #We need to flatten the input size before feeding it to neural network
    def __init__(self, input_size=3*32*32, output_size=10):
        super().__init__()
        self._to_linear = None    
        self.fc1 = nn.Linear(input_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, output_size)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=1)

In [ ]:
#creating a variable "artificial_nn" to call ANN() class
torch.manual_seed(80)
ann = ANN()
ann

In [ ]:
#Creation of variable to calculate loss and to specify optimizer for ANN

cal_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(ann.parameters(), lr=0.01)

ANN Model Training

In [ ]:
#creating empty list to calculate loss during training and to keep track of correct predictions
loss_train = []
loss_test = []
correct_train = []
correct_test  = []

epochs = 5
for i in range(epochs):
   
    trn_corr = 0
    tst_corr = 0
    batch_corr = 0
    

    for batch_iter, (X_train, y_train) in enumerate(train_loader):
      batch_iter = batch_iter + 1

      y_pred = ann(X_train.view(100, -1))
      loss = cal_loss(y_pred, y_train)
      
      predicted = torch.max(y_pred.data, 1)[1]
      batch_corr = (predicted == y_train).sum()
      trn_corr = trn_corr + batch_corr
      
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if batch_iter % 100 == 0:
        
        accuracy = trn_corr.item()*100 / (100*batch_iter)
        print( f'epoch: {i} batch {batch_iter} loss:{loss.item()} accuracy:{accuracy} ')
    
    loss_train.append(loss)
    correct_train.append(trn_corr)
     with torch.no_grad():
      
      for batch_iter, (X_test, y_test) in enumerate(test_loader):
        
        y_val = ann(X_test.view(500, -1))
        
        predicted = torch.max(y_val.data, 1)[1]
        tst_corr = tst_corr + (predicted == y_test).sum()
     
    loss = cal_loss(y_val,y_test)
    loss_test.append(loss)
    correct_test.append(tst_corr)

ANN Model Testing

In [ ]:
#Loading all 10000 test images at once for prediction
load_testdata = DataLoader(test_set, batch_size=10000, shuffle=False)

with torch.no_grad():
  correct = 0
  for X_test, y_test in load_testdata: 
    y_val = ann(X_test.view(len(X_test),-1))
    predicted = torch.max(y_val,1)[1]
    correct = correct + (predicted == y_test).sum()
    accuracy_ann = 100 * correct.item()/len(test_set)
print(f'Accuracy on test set by ANN: {accuracy_ann}')

Building CNN Model

In [ ]:
# define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # fully connected layers
        self.fc1 = nn.Linear(64 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, 10)
        # dropout
        self.dropout = nn.Dropout(p=.5)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flattening
        x = x.view(-1, 64 * 4 * 4)
        # fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

# create a variable "cnn" for CNN() class
cnn = CNN()
cnn

In [ ]:
#Creation of variable to calculate loss and to specify optimizer for CNN

cal_loss_cnn = nn.CrossEntropyLoss()
optimizer_cnn = torch.optim.SGD(cnn.parameters(), lr=.001)

CNN Model Training

In [ ]:
# number of epochs to train the model
n_epochs = [*range(30)]
#List to store loss to visualize
train_losslist = []
valid_losslist = []
valid_loss_min = np.Inf # track change in validation loss

for epoch in n_epochs:

    # keeping track of train and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    cnn.train()
    for data, target in train_loader_cnn:
        # clearing gradient
        optimizer_cnn.zero_grad()
        output = cnn(data)
        loss = cal_loss_cnn(output, target)
        loss.backward()
        optimizer_cnn.step()
        train_loss = train_loss + loss.item()*data.size(0)
    cnn.eval()
    for data, target in valid_loader_cnn:
        output = cnn(data)
        loss = cal_loss_cnn(output, target)
        valid_loss = valid_loss + loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader_cnn.dataset)
    valid_loss = valid_loss/len(valid_loader_cnn.dataset)
    train_losslist.append(train_loss)
    valid_losslist.append(valid_loss)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))

plt.plot(n_epochs, train_losslist)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Performance of Model")
plt.show()

CNN Model Testing

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
# testing on test data
cnn.eval()
for data, target in test_loader_cnn:
    output = cnn(data)
    _, pred = torch.max(output, 1)    
    # comparing true label with predictions
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())
    # calculating accuracy
    batch_size = 20
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] = class_correct[label] + correct[i].item()
        class_total[label] = class_total[label] + 1
accuracy_cnn = 100 * np.sum(class_correct) / np.sum(class_total)
print(f'\nAccuracy on test set by CNN: {accuracy_cnn}%')

In [ ]:
# comparing accuracies of both the models
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_ylabel('Percentage%')
ax.set_xlabel('Models')
model_name = ['ANN', 'CNN']
accu = [accuracy_ann, accuracy_cnn]
ax.bar(model_name,accu)
plt.ylim(0,100)
plt.show()